In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/donnees_battery.csv", index_col="datetime", parse_dates=True)
print("les données ont été chargées avec succès !")

In [ ]:
df["hour"] = df.index.hour
df["tarif"] = np.where((df["hour"] >= 22) | (df["hour"] < 6), 0.12, 0.18) # Tarif de 0.18€/kWh de 6h à 22h, sinon 0.12€/kWh
df["cout_import"] = df["import_grid"]* df["tarif"]
df["revenu_export"] = df["export_grid"] * 0.10 # Supposons un tarif de rachat de 0.10€/kWh pour l'export
df["cout_net"] = df["cout_import"] - df["revenu_export"]

In [ ]:
cout_annuel = df["cout_net"].sum()
cout_sans_pv = (df["power_kW"] * df["tarif"]).sum()
cout_sans_batterie = (df["soutirage_kW"] * df["tarif"]).sum()
print(f"Coût annuel net : {cout_annuel:.2f} €")
print(f"Coût annuel sans PV : {cout_sans_pv:.2f} €")
print(f"Coût annuel sans batterie : {cout_sans_batterie:.2f} €")

In [ ]:
investissement_batterie = 50000 # Coût d'investissement pour la batterie
economie_annuelle = cout_sans_batterie - cout_annuel

roi = investissement_batterie / economie_annuelle if economie_annuelle > 0 else float('inf')
print(f"Économie annuelle : {economie_annuelle:.2f} €")
print(f"Retour sur investissement : {roi:.1f} ans")